In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Connected to MongoDB successfully.\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Dash app running on http://127.0.0.1:8050/\n"
     ]
    }
   ],
   "source": [
    "# Setup the Jupyter version of Dash\n",
    "from jupyter_dash import JupyterDash\n",
    "\n",
    "# Configure the necessary Python module imports\n",
    "import dash\n",
    "import dash_leaflet as dl\n",
    "from dash import dcc, html, dash_table\n",
    "from dash.dependencies import Input, Output\n",
    "import plotly.express as px\n",
    "import pandas as pd\n",
    "import os\n",
    "import base64\n",
    "\n",
    "# Import the CRUD module\n",
    "from main import AnimalShelter\n",
    "\n",
    "###########################\n",
    "# Data Manipulation / Model\n",
    "###########################\n",
    "# Credentials\n",
    "USERNAME = os.getenv(\"MONGO_USER\", \"aacuser\")\n",
    "PASSWORD = os.getenv(\"MONGO_PASS\", \"strongpassword123\")\n",
    "HOST = os.getenv(\"MONGO_HOST\", \"nv-desktop-services.apporto.com\")\n",
    "PORT = int(os.getenv(\"MONGO_PORT\", \"31580\"))\n",
    "DB_NAME = \"AAC\"\n",
    "COLLECTION = \"animals\"\n",
    "\n",
    "# Initialize the database connection\n",
    "shelter = AnimalShelter(USERNAME, PASSWORD, HOST, PORT, DB_NAME, COLLECTION)\n",
    "\n",
    "# Read the initial data for the dashboard\n",
    "df = pd.DataFrame.from_records(shelter.read({}))\n",
    "\n",
    "# Ensure '_id' field does not break DataTable rendering\n",
    "if '_id' in df.columns:\n",
    "    df.drop(columns=['_id'], inplace=True)\n",
    "\n",
    "############################\n",
    "# Load Grazioso Salvare Logo\n",
    "############################\n",
    "image_path = \"./Grazioso Salvare Logo.png\"\n",
    "\n",
    "# Encode the image\n",
    "with open(image_path, \"rb\") as image_file:\n",
    "    encoded_image = base64.b64encode(image_file.read()).decode()\n",
    "\n",
    "#########################\n",
    "# Dashboard Layout / View\n",
    "#########################\n",
    "app = JupyterDash(__name__)\n",
    "\n",
    "app.layout = html.Div([\n",
    "    html.Center(html.Img(\n",
    "        src=f\"data:image/png;base64,{encoded_image}\",\n",
    "        style={\"width\": \"200px\", \"height\": \"auto\"}\n",
    "    )),\n",
    "    html.Center(html.B(html.H1(\"Chris' SNHU CS-340 Dashboard\"))),\n",
    "    html.Hr(),\n",
    "\n",
    "    # Add Buttons to Filter by Rescue Type\n",
    "    html.Div([\n",
    "        html.Button(\"Water Rescue\", id=\"btn-water\", n_clicks=0, style={'margin': '5px'}),\n",
    "        html.Button(\"Mountain/Wilderness Rescue\", id=\"btn-mountain\", n_clicks=0, style={'margin': '5px'}),\n",
    "        html.Button(\"Disaster/Individual Tracking\", id=\"btn-disaster\", n_clicks=0, style={'margin': '5px'}),\n",
    "        html.Button(\"Reset\", id=\"btn-reset\", n_clicks=0, style={'margin': '5px'})\n",
    "    ], style={'textAlign': 'center', 'margin-bottom': '10px'}),\n",
    "\n",
    "    html.Hr(),\n",
    "    dash_table.DataTable(\n",
    "        id='datatable-id',\n",
    "        columns=[{\"name\": i, \"id\": i, \"deletable\": False, \"selectable\": True} for i in df.columns],\n",
    "        data=df.to_dict('records'),\n",
    "        style_table={'overflowX': 'auto'},\n",
    "        row_selectable='single',\n",
    "        filter_action='native',\n",
    "        sort_action='native',\n",
    "        page_size=10,\n",
    "        selected_rows=[0],\n",
    "    ),\n",
    "    html.Br(),\n",
    "    html.Div(className='row', style={'display': 'flex'}, children=[\n",
    "        html.Div(id='graph-id', className='col s12 m6'),\n",
    "        html.Div(id='map-id', className='col s12 m6'),\n",
    "    ]),\n",
    "])\n",
    "\n",
    "#############################################\n",
    "# Interaction Between Components / Controller\n",
    "#############################################\n",
    "\n",
    "# Callback to filter data using MongoDB aggregation\n",
    "@app.callback(\n",
    "    Output('datatable-id', 'data'),\n",
    "    [Input('btn-water', 'n_clicks'),\n",
    "     Input('btn-mountain', 'n_clicks'),\n",
    "     Input('btn-disaster', 'n_clicks'),\n",
    "     Input('btn-reset', 'n_clicks')]\n",
    ")\n",
    "def update_dashboard(n_water, n_mountain, n_disaster, n_reset):\n",
    "    ctx = dash.callback_context\n",
    "\n",
    "    button_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else 'btn-reset'\n",
    "\n",
    "    if button_id == 'btn-reset':\n",
    "        all_data = shelter.read({})\n",
    "        df = pd.DataFrame.from_records(all_data)\n",
    "        if '_id' in df.columns: df['_id'] = df['_id'].astype(str)\n",
    "        return df.to_dict('records')\n",
    "\n",
    "    # Define breeds by rescue type\n",
    "    breeds_map = {\n",
    "        \"btn-water\": [\"Labrador Retriever\", \"Newfoundland\", \"Chesapeake Bay Retriever\", \"Turkish Van\", \"Maine Coon\"],\n",
    "        \"btn-mountain\": [\"German Shepherd\", \"Border Collie\", \"Bernese Mountain Dog\", \"Norwegian Forest Cat\", \"Siberian Cat\"],\n",
    "        \"btn-disaster\": [\"Bloodhound\", \"Belgian Malinois\", \"Doberman Pinscher\", \"Bengal\", \"Abyssinian\"]\n",
    "    }\n",
    "    \n",
    "    target_breeds = breeds_map.get(button_id)\n",
    "    if not target_breeds:\n",
    "        return []\n",
    "\n",
    "    # Build and execute the aggregation pipeline\n",
    "    pipeline = [{'$match': {'breed': {'$in': target_breeds}}}]\n",
    "    filtered_data = shelter.aggregate(pipeline)\n",
    "    \n",
    "    df = pd.DataFrame.from_records(filtered_data)\n",
    "    if '_id' in df.columns: df['_id'] = df['_id'].astype(str)\n",
    "    \n",
    "    return df.to_dict('records')\n",
    "\n",
    "# Callback to update pie chart\n",
    "@app.callback(\n",
    "    Output('graph-id', \"children\"),\n",
    "    Input('datatable-id', \"derived_virtual_data\")\n",
    ")\n",
    "def update_graph(viewData):\n",
    "    if not viewData:\n",
    "        return []\n",
    "    \n",
    "    dff = pd.DataFrame.from_dict(viewData)\n",
    "    return dcc.Graph(figure=px.pie(dff, names=\"breed\", title=\"Animal Breeds Distribution\"))\n",
    "\n",
    "# Callback to update geolocation map\n",
    "@app.callback(\n",
    "    Output('map-id', \"children\"),\n",
    "    [Input('datatable-id', \"derived_virtual_data\"),\n",
    "     Input('datatable-id', \"derived_virtual_selected_rows\")]\n",
    ")\n",
    "def update_map(viewData, index):\n",
    "    if not viewData:\n",
    "        return dl.Map(center=[30.75, -97.48], zoom=4, children=[dl.TileLayer()])\n",
    "    \n",
    "    dff = pd.DataFrame.from_dict(viewData)\n",
    "    row_idx = index[0] if index else 0\n",
    "    \n",
    "    if row_idx >= len(dff):\n",
    "        return dl.Map(center=[30.75, -97.48], zoom=4, children=[dl.TileLayer()])\n",
    "    \n",
    "    animal = dff.iloc[row_idx]\n",
    "    \n",
    "    return dl.Map(\n",
    "        style={'width': '1000px', 'height': '500px'},\n",
    "        center=[animal['location_lat'], animal['location_long']],\n",
    "        zoom=10,\n",
    "        children=[\n",
    "            dl.TileLayer(),\n",
    "            dl.Marker(\n",
    "                position=[animal['location_lat'], animal['location_long']],\n",
    "                children=[\n",
    "                    dl.Tooltip(animal['breed']),\n",
    "                    dl.Popup([\n",
    "                        html.H2(animal.get('name', 'Unknown Name')),\n",
    "                        html.P(f\"Breed: {animal['breed']}\")\n",
    "                    ])\n",
    "                ]\n",
    "            )\n",
    "        ]\n",
    "    )\n",
    "\n",
    "# Run the app\n",
    "if __name__ == '__main__':\n",
    "    app.run_server(debug=True)\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

Connected to MongoDB successfully.
Dash app running on http://127.0.0.1:13526/
